# 2020 Önálló kutatási feladat

In [2]:
import pandas as pd
import re

## Stock data preprocessing

In [3]:
stock = pd.read_csv('..\Adatok\stock-data\SP500.csv')

stock = stock.drop(columns=['High','Low','Adj Close','Volume'])
stock['Date'] = pd.to_datetime(stock['Date']).dt.date

def Change(row):
    return (row['Close']-row['Open'])

stock['Change'] = stock.apply (lambda row: Change(row), axis=1)

def label(row, threshold): 
    if abs(row['Change']) > threshold:
        if row['Change'] > 0:
            return int(1)
        else:
            return int(-1)
    else:
        return int(0)


def generate_data(sensitivity):
    stock['label'] = stock.apply (lambda row: label(row, sensitivity), axis=1)
    stock_ready = stock.drop(columns=['Open', 'Close', 'Change'])
    stock_ready = stock_ready[stock_ready.label != 0]
    return stock_ready 

stock

,Date,Open,Close,Change
0,2000-01-03,1469.250000,1455.219971,-14.030029
1,2000-01-04,1455.219971,1399.420044,-55.799927
2,2000-01-05,1399.420044,1402.109985,2.689941
3,2000-01-06,1402.109985,1403.449951,1.339966
4,2000-01-07,1403.449951,1441.469971,38.020020
...,...,...,...,...
5164,2020-07-14,3141.110107,3197.520020,56.409913
5165,2020-07-15,3225.979980,3226.560059,0.580079
5166,2020-07-16,3208.360107,3215.570068,7.209961
5167,2020-07-17,3224.209961,3224.729980,0.520019


### Itt tudjuk a *generate_data* függvény paraméterével állítani, hogy milyen mértékű változást veszünk figyelembe.

In [4]:
stock_labeled = generate_data(2)
stock_labeled

,Date,label
0,2000-01-03,-1
1,2000-01-04,-1
2,2000-01-05,1
4,2000-01-07,1
5,2000-01-10,1
...,...,...
5162,2020-07-10,1
5163,2020-07-13,-1
5164,2020-07-14,1
5166,2020-07-16,1


## Twitteres adatok preprocessing

In [5]:
#Obama
tweets_dem = pd.read_csv('..\Adatok\obama\obama-tweets.csv')
tweets_dem = tweets_dem.drop(columns=['Username', 'Tweet Link', 'Retweets', 'Likes', 'TweetImageUrl', 'Image'])
tweets_dem['Date'] = pd.to_datetime(tweets_dem['Date'], format='%Y/%m/%d_%H:%M')
tweets_dem['Date'] = tweets_dem['Date'].dt.date
tweets_dem = tweets_dem[tweets_dem.Date < pd.to_datetime('2017-01-21')]
tweets_dem= tweets_dem.sort_index(ascending=False, axis=0)
tweets_dem.reset_index(drop=True,inplace=True)
tweets_dem.insert(2, "President", "Obama", allow_duplicates=False) #rájöttem, hogy ez redundáns, mert a dátum alapján tudjuk

tweets_dem

,Date,Tweet-text,President
0,2012-11-05,"President Obama tells the story of ""Fired up! ...",Obama
1,2012-11-06,Election Day is here! Confirm your polling pla...,Obama
2,2012-11-06,It’s Election Day! This is your last chance to...,Obama
3,2012-11-06,At the final rally of his final campaign last ...,Obama
4,2012-11-06,25 reasons that 25 people are voting for Presi...,Obama
...,...,...,...
6673,2016-11-04,Let's keep working to keep our economy on a be...,Obama
6674,2016-11-05,In the weekly address President Obama discusse...,Obama
6675,2017-01-10,Tonight President Obama reflects on eight year...,Obama
6676,2017-01-20,Hi everybody! Back to the original handle. Is ...,Obama


In [6]:
#Trump
tweets_gop = pd.read_csv('../Adatok/trump/trump-tweets.csv')
tweets_gop = tweets_gop.drop(columns=['id', 'link', 'retweets', 'favorites', 'mentions', 'hashtags'])
tweets_gop['date'] = pd.to_datetime(tweets_gop['date'])
tweets_gop['date'] = tweets_gop['date'].dt.date
tweets_gop = tweets_gop[['date', 'content']]
tweets_gop = tweets_gop[ pd.to_datetime('2017-01-21') <= tweets_gop.date]
tweets_gop.reset_index(drop=True,inplace=True)
tweets_gop.columns = ["Date","Tweet-text"]
tweets_gop.insert(2, "President", "Trump", allow_duplicates=False) #rájöttem, hogy ez redundáns, mert a dátum alapján tudjuk

tweets_gop

,Date,Tweet-text,President
0,2017-01-21,A fantastic day and evening in Washington D.C....,Trump
1,2017-01-22,Had a great meeting at CIA Headquarters yester...,Trump
2,2017-01-22,Watched protests yesterday but was under the i...,Trump
3,2017-01-22,"Wow, television ratings just out: 31 million p...",Trump
4,2017-01-22,Peaceful protests are a hallmark of our democr...,Trump
...,...,...,...
12086,2020-06-17,Joe Biden was a TOTAL FAILURE in Government. H...,Trump
12087,2020-06-17,Will be interviewed on @ seanhannity tonight a...,Trump
12088,2020-06-17,pic.twitter.com/3lm1spbU8X,Trump
12089,2020-06-17,pic.twitter.com/vpCE5MadUz,Trump


## Táblák joinolása

In [7]:
joined_data = pd.concat([tweets_dem,tweets_gop],ignore_index=True)
#joined_data = joined_data.merge(stock_labeled, how='inner', left_on='Date', right_on='Date') --> fölösleges itt mergelni, ha később úgyis összevonunk még sorokat
joined_data.columns = ['date','text','president']
joined_data

,date,text,president
0,2012-11-05,"President Obama tells the story of ""Fired up! ...",Obama
1,2012-11-06,Election Day is here! Confirm your polling pla...,Obama
2,2012-11-06,It’s Election Day! This is your last chance to...,Obama
3,2012-11-06,At the final rally of his final campaign last ...,Obama
4,2012-11-06,25 reasons that 25 people are voting for Presi...,Obama
...,...,...,...
18764,2020-06-17,Joe Biden was a TOTAL FAILURE in Government. H...,Trump
18765,2020-06-17,Will be interviewed on @ seanhannity tonight a...,Trump
18766,2020-06-17,pic.twitter.com/3lm1spbU8X,Trump
18767,2020-06-17,pic.twitter.com/vpCE5MadUz,Trump


## Adatok tisztítása

In [8]:
def tags(row): #Twitter tag-elést szűrő fv
    return re.sub(r'@ ?(\w){1,15}', " ", row['text'] )

def picture(row): #képek elé space-t tevő fv Azért kell, mert nem tesznek spacet a kép és az utolsó szó közé, ezért az utolsó szó is törlődik a linkszűrésnél
    return re.sub(r'pic.twitter.com/', " pic.twitter.com/", row['text'] )

def insertspace(row):
    return re.sub(r'(https?:\/\/)', ' https://', row['text'] )
    
def urls(row): # linkeket szűrő fv
    return re.sub(r'(https?:\/\/)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&\/=]*)', " ", row['text'] )

joined_data['text'] = joined_data.apply (lambda row: picture(row), axis=1)
joined_data['text'] = joined_data.apply (lambda row: insertspace(row), axis=1)
joined_data['text'] = joined_data.apply (lambda row: tags(row), axis=1)
joined_data['text'] = joined_data.apply (lambda row: urls(row), axis=1)
joined_data = joined_data[joined_data.text != "  " ]
joined_data = joined_data[joined_data.text != "  …" ]
joined_data.reset_index(drop=True,inplace=True)

joined_data

,date,text,president
0,2012-11-05,"President Obama tells the story of ""Fired up! ...",Obama
1,2012-11-06,Election Day is here! Confirm your polling pla...,Obama
2,2012-11-06,It’s Election Day! This is your last chance to...,Obama
3,2012-11-06,At the final rally of his final campaign last ...,Obama
4,2012-11-06,25 reasons that 25 people are voting for Presi...,Obama
...,...,...,...
18335,2020-06-16,True! …,Trump
18336,2020-06-16,A GREAT woman. Her son is looking down from he...,Trump
18337,2020-06-16,96% Approval Rating in the Republican Party. T...,Trump
18338,2020-06-17,Joe Biden was a TOTAL FAILURE in Government. H...,Trump


In [9]:
cjd = pd.DataFrame() #concated joined data a.k.a. az azonos dátumú sorok textjei össze vannak concat-olva
cjd['date'] = joined_data.loc[:,'date'].drop_duplicates()
cjd.reset_index(drop=True,inplace=True)

string_list = []
for i in cjd.to_numpy():
    temp_string = ""
    for j in joined_data.loc[joined_data['date'] == i[0]].to_numpy():
        temp_string = temp_string + " " + j[1]
    temp_string = temp_string.replace('\xa0',' ').replace('…',' ')
    temp_string = ' '.join(temp_string.split())
    string_list.append(temp_string)
cjd['text'] = string_list

cjd = cjd.merge(stock_labeled, how='inner', left_on='date', right_on='Date').drop('Date',axis=1)
cjd

,date,text,label
0,2012-11-05,"President Obama tells the story of ""Fired up! ...",1
1,2012-11-06,Election Day is here! Confirm your polling pla...,1
2,2012-11-07,Thank you. President Obama in his victory spee...,-1
3,2012-11-08,The definition of hope is you still believe ev...,-1
4,2012-11-09,What Bobby Kennedy called the ripples of hope ...,1
...,...,...,...
1548,2020-06-11,Our great National Guard Troops who took care ...,-1
1549,2020-06-12,People have no idea how Fake the Lamestream Me...,-1
1550,2020-06-15,I’ve done more in less than 4 years than Biden...,1
1551,2020-06-16,Wow! May retail sales show biggest one-month i...,-1


## Kiírás fájlba

In [11]:
cjd.to_excel('../Adatok/tisztitott_adat.xlsx')
ncjd = joined_data.merge(stock_labeled, how='inner', left_on='date', right_on='Date').drop('Date',axis=1)
ncjd.to_excel('../Adatok/non_cdj.xlsx')